In [1]:
import coffea
from git import Repo
import awkward as ak
import dask_awkward as dak
mod = "Prayag Yadav"
local_repo = Repo(path='coffea')
local_branch = local_repo.active_branch.name
print("_______________________________________")
print("\tCurrent Configuration")
print("---------------------------------------")
print("Coffea Version: ", coffea.__version__)
print("Branch: \t", local_branch)
print("Modified by: \t", mod)
print("_______________________________________")


_______________________________________
	Current Configuration
---------------------------------------
Coffea Version:  0.1.dev3583+ge06c4b8
Branch: 	 master
Modified by: 	 Prayag Yadav
_______________________________________


In [2]:
from coffea.nanoevents import NanoEventsFactory, FCCSchema, FCC
test_file = '../../../coffea-fcc-analyses/data/p8_ee_ZH_ecm240/events_082532938.root'
events = NanoEventsFactory.from_root(
    test_file+":events",
    entry_stop=100,
    schemaclass= FCC.get_schema(version="latest", zip_missing=False),
    delayed = False
).events()

/home/prayag/coffeafcc/development/coffea/src/coffea/nanoevents/methods/fcc.py:7: FutureWarning: In version 2025.1.0 (target date: 2024-12-31 11:59:59-06:00), this will be an error.
To raise these warnings as errors (and get stack traces to find out where they're called), run
    import warnings
    warnings.filterwarnings("error", module="coffea.*")
after the first `import coffea` or use `@pytest.mark.filterwarnings("error:::coffea.*")` in pytest.
Issue: coffea.nanoevents.methods.vector will be removed and replaced with scikit-hep vector. Nanoevents schemas internal to coffea will be migrated. Otherwise please consider using that package!.
  from coffea.nanoevents.methods import base, vector


In [3]:
events.fields

['AllMuonidx0',
 'EFlowNeutralHadron',
 'EFlowNeutralHadronidx0',
 'EFlowNeutralHadronidx1',
 'EFlowNeutralHadronidx2',
 'EFlowPhoton',
 'EFlowPhotonidx0',
 'EFlowPhotonidx1',
 'EFlowPhotonidx2',
 'EFlowTrack',
 'EFlowTrackidx0',
 'EFlowTrackidx1',
 'Electronidx0',
 'Jet',
 'Jetidx0',
 'Jetidx1',
 'Jetidx2',
 'Jetidx3',
 'Jetidx4',
 'Jetidx5',
 'MCRecoAssociations',
 'MCRecoAssociationsidx0',
 'MCRecoAssociationsidx1',
 'MissingET',
 'MissingETidx0',
 'MissingETidx1',
 'MissingETidx2',
 'MissingETidx3',
 'MissingETidx4',
 'MissingETidx5',
 'Muonidx0',
 'Particle',
 'ParticleIDs',
 'Particleidx0',
 'Particleidx1',
 'Photonidx0',
 'ReconstructedParticles',
 'ReconstructedParticlesidx0',
 'ReconstructedParticlesidx1',
 'ReconstructedParticlesidx2',
 'ReconstructedParticlesidx3',
 'ReconstructedParticlesidx4',
 'ReconstructedParticlesidx5']

In [4]:
events.ReconstructedParticles.match_collection(events.Muonidx0).absolute_mass

<Array [[], [], [], ..., [], [0.00276, 0.00781], []] type='100 * var * float32'>